In [34]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from copy import deepcopy

In [35]:
df = pd.read_json("contacts.json")

In [36]:
df.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,


In [37]:
# create a new dictionary where the key is id, and the value is a list. this list is mutable
default = defaultdict(set)
default

defaultdict(set, {})

In [38]:
# improve the default dictionary with email column first
dfEmail = df[df['Email']!=""].drop(['Phone','Contacts','OrderId'], axis=1)
dfEmail.head()

,Id,Email
0,0,gkzAbIy@qq.com
3,3,mdllpYmE@gmail.com
6,6,hESiBgYTxMnj@hotmail.com
7,7,hqyunTiaXfIjhO@yahoo.com
8,8,XGUjcSiDpFKWEUCs@qq.com


In [39]:
# improve the default dictionary with Phone column first
dfPhone = df[df['Phone']!=""].drop(['Email','Contacts','OrderId'], axis=1)
dfPhone.head()

,Id,Phone
1,1,329442681752
2,2,9125983679
4,4,300364407
5,5,840113148
8,8,223604620644


In [40]:
# improve the default dictionary with  OrderId column first
dfOrderId = df[df['OrderId']!=""].drop(['Email','Contacts','Phone'], axis=1)
dfOrderId.head()

,Id,OrderId
1,1,vDDJJcxfLtSfkooPhbYnJdxov
3,3,bHquEnCbbsGLqllwryxPsNOxa
7,7,vxjYZtMUvNhtEBDruzQfjpsmX
9,9,IqewrMXVTJqSsVklWFfOpdRNI
10,10,OHRFAXqbAzyxljoUDaOErwtrg


In [41]:
# update the default dictionary first with email, then phone, then OrderId
for key, group in tqdm(dfEmail.groupby("Email")):
    # access pandas groupby
    ids = list(group['Id'])
    #print(ids)
    for id in ids:
        #print(id,ids)
        default[id].update(ids) # not same as default[id] = ids as it will be a list, not set
        
        
# https://docs.python.org/3/library/collections.html#collections.defaultdict 
# https://www.programiz.com/python-programming/methods/dictionary/update
    

100%|██████████████████████████████████████████| 249156/249156 [00:33<00:00, 7442.05it/s]


In [42]:
# update the default dictionary first with email, then phone, then OrderId
for key, group in tqdm(dfPhone.groupby("Phone")):
    # access pandas groupby
    ids = list(group['Id'])
    #print(ids)
    for id in ids:
        #print(id,ids)
        default[id].update(ids) # not same as default[id] = ids as it will be a list, not set

100%|██████████████████████████████████████████| 190677/190677 [00:24<00:00, 7796.96it/s]


In [43]:
# update the default dictionary first with email, then phone, then OrderId
for key, group in tqdm(dfOrderId.groupby("OrderId")):
    # access pandas groupby
    ids = list(group['Id'])
    #print(ids)
    for id in ids:
        #print(id,ids)
        default[id].update(ids) # not same as default[id] = ids as it will be a list, not set

100%|██████████████████████████████████████████| 189302/189302 [00:22<00:00, 8510.69it/s]


In [44]:
# try to link between Email, Phone and OrderId, for each id, increase their size if they are linked
default_copy = deepcopy(default)

for key,items in default_copy.items():
    # for each key(where some are not crossed between email,phone and Orderid), 
    # they have a set of other keys sharing same email/phone/orderid or more than one
    union = set()
    # copy the empty union
    union_before = deepcopy(union)
    # update the union with items, which are the values of each default_copy per id
    union.update(items)
    # Allow "union" to keep updating until it stabilise
    #
    while union_before != union.copy():
        union_before = union.copy()
        #print(union)
        #print("-----")
        for id in union.copy():
            union.update(default[id]) # for each id in default, they have a set. keep updating the union(per id)
    default[key].update(union)

In [45]:
# create contact
# empty dict called contact
contact = defaultdict()

# if id is already looked at, no need to waste resource look again
for i in tqdm(range(len(df))):
    if i in contact:
        continue
     
    # per same person, (one default set) we find sum of all their contacts
    contactPerId = 0
    for index in default[i]: 
        contactPerId += df.iloc[index]['Contacts']
        
    # must separate this for loop from previous because the previous for loop has not added finish all the contacts per person
    for index in default[i]:
        contact[index] = str(contactPerId)
    
    #print("====")

    
        

#https://realpython.com/python-defaultdict/#:~:text=The%20Python%20defaultdict%20type%20behaves,handling%20missing%20keys%20in%20dictionaries.

100%|██████████████████████████████████████████| 500000/500000 [01:26<00:00, 5753.30it/s]


In [46]:
# join all the values in the set of ticket_trace into one string
joined_string = defaultdict()
for id,items in default.items():
    items_sorted = sorted(items)
    list_of_strings = [str(s) for s in items_sorted]
    joined_string[id] = "-".join(list_of_strings)
    


In [55]:
# Join dictionary "contact","default" together via id 
dd = defaultdict(list)

for d in (joined_string,contact): # you can list as many input dicts as you want here
    for key, value in d.items():
        dd[key].append(value)
        
for id, value in dd.items():
    dd[id] = ', '.join(value)
    
dd

defaultdict(list,
            {112387: '112387-470075, 4',
             470075: '112387-470075, 4',
             338154: '338154, 2',
             426495: '426495, 2',
             37868: '37868-126317-308856, 7',
             137956: '137956, 1',
             494574: '78833-407938-494574, 6',
             19504: '19504-179125, 3',
             179125: '19504-179125, 3',
             299505: '51764-73472-216393-299505-310113-435438-475822-482411, 21',
             299171: '299171, 0',
             386346: '386346, 4',
             99859: '99859, 3',
             326760: '326760, 4',
             76239: '76239, 2',
             462902: '462902, 4',
             487374: '487374, 2',
             87543: '87543-414465, 3',
             20825: '20825-337532, 5',
             337532: '20825-337532, 5',
             305394: '305394, 2',
             362: '362, 3',
             153039: '153039, 4',
             221848: '221848-330517, 4',
             330517: '221848-330517, 4',
             2

In [56]:
# mapping the joined dictionary of "default" and "contact" 
df_new = pd.Series(dd).to_frame()
# make a new column which is the index column
df_new['ticket_id'] = df_new.index
# rename the contacts and default column 
df_new.rename(columns={df_new.columns[0]:'ticket_trace/contact'}, inplace=True)
df_new = df_new[['ticket_id','ticket_trace/contact']]
# sort values by ticket_id
df_new.sort_values(by=['ticket_id'], inplace = True)
df_new

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
...,...,...
499995,499995,"499995, 2"
499996,499996,"499996, 4"
499997,499997,"499997, 2"
499998,499998,"121111-499998, 5"


In [57]:
df_new['ticket_trace/contact'].nunique()

291904

In [58]:
df_new.to_csv("jy_output.csv")

In [59]:
# compare the results against the correct results
df_correct = pd.read_csv("result.csv")
df_correct.head()

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"


In [60]:

points = 0
for i in range(len(df_correct)):
    if df_new['ticket_trace/contact'][i] == df_correct['ticket_trace/contact'][i]:
        
        points += 1
        
    else:
        points += 0
accuracy = points/len(df_correct)

print(f"Accuracy score: {accuracy}")

Accuracy score: 1.0


In [61]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 2 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   ticket_id             500000 non-null  int64 
 1   ticket_trace/contact  500000 non-null  object
dtypes: int64(1), object(1)
memory usage: 27.6+ MB


In [62]:
df_correct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 2 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   ticket_id             500000 non-null  int64 
 1   ticket_trace/contact  500000 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.6+ MB


# NOTES!

In [19]:
# "ADD" method only for integers
s = [('red', 1), ('blue', 2), ('red', 3), ('blue', 4), ('red', 1), ('blue', 4)]
d = defaultdict(set)
for k, v in s:
    d[k].add(v)
d.items()

dict_items([('red', {1, 3}), ('blue', {2, 4})])

In [20]:
# "Update" method only for string or others
s = [('red', '1'), ('blue', '2'), ('red', '3'), ('blue', '4'), ('red', '1'), ('blue', '4')]
d = defaultdict(set)
for k, v in s:
    d[k].update(v)
d.items()

# default dict if id not exist will then just be {} right? since it was set to be defaultdict(set): Correct

dict_items([('red', {'3', '1'}), ('blue', {'4', '2'})])

## From ChenXi

In [21]:
# take union of all the ids for all contact mthods because "email" sets have no knowledge of "Phone" sets etc
id_trace_copy = deepcopy(default)
for key, value in id_trace_copy.items():
    union_set = set()
    union_size_before = len(union_set)
    union_set.update(value)
    #print(union_set)
    
    
    while (len(union_set) != union_size_before):
        #print(union_size_before)
        union_size_before = len(union_set)
        #print(union_size_before)
        for id in union_set.copy():
            union_set.update(default[id])
            
    
    default[key].update(union_set)

In [22]:
contact = defaultdict()
for i in tqdm(range(len(df))):
    if i in contact:
        continue
        
    total_contact = 0
    for id in default[i]:
        total_contact += df.loc[id, "Contacts"]
    for id in default[i]:
        contact[id] = total_contact

100%|█████████████████████████████████████████| 500000/500000 [00:09<00:00, 54371.46it/s]


In [23]:
result_list = []
for i in tqdm(range(len(df))):
    sorted_ids = sorted(default[i])
    ticket_trace = '-'.join(str(id) for id in sorted_ids)
    ticket_trace_contact = ticket_trace + ", " + str(contact[i])
    result_list.append([i, ticket_trace_contact])

result_df = pd.DataFrame(result_list, columns=["ticket_id", "ticket_trace/contact"])

100%|████████████████████████████████████████| 500000/500000 [00:03<00:00, 140267.60it/s]


In [24]:
result_df['ticket_trace/contact'].nunique()

291904

# Testing code to see output

In [25]:
test_default = {1:{1,2,943},2:{1,2,300,400,943},943:{1,2,943},300:{2,300,400},400:{2,943}}
# take union of all the ids for all contact mthods because "email" sets have no knowledge of "Phone" sets etc
id_trace_copy = deepcopy(test_default)
for key, value in id_trace_copy.items():
    union_set = set()
    print(union_set)
    union_size_before = len(union_set)
    union_set.update(value)
    print(union_set)
    
    while (len(union_set) != union_size_before):
        union_size_before = len(union_set)
        for id in union_set.copy():
            union_set.update(test_default[id])
    
    test_default[key].update(union_set)
test_default

set()
{1, 2, 943}
set()
{400, 1, 2, 300, 943}
set()
{1, 2, 943}
set()
{400, 2, 300}
set()
{2, 943}


{1: {1, 2, 300, 400, 943},
 2: {1, 2, 300, 400, 943},
 943: {1, 2, 300, 400, 943},
 300: {1, 2, 300, 400, 943},
 400: {1, 2, 300, 400, 943}}